In [1]:
import time
import numpy as np
import math
import itertools
from Crypto.Util import number
import Crypto.Random as Random
from Crypto.Hash import SHA256

# OLH

In [2]:
class Hasher():
    def __init__(self, seed, d):
        self.seed = seed
        self.hash = SHA256.new(data=bytes(seed))
        self.d = d
        
    def initialize(self, seed):
        self.seed = seed
        self.hash = SHA256.new(data=bytes(seed))

    def encode(self, data):
        self.hash.update(bytes(data))
        hash_value = int(self.hash.hexdigest(), 16) % self.d
        self.hash = SHA256.new(data=bytes(self.seed))
        return hash_value

def buildParams(epsilon, width, d):
    l, n = decideRatio(epsilon, d, width)
    assert (n-l) % d == 0, "Invalied combination, n, l, d"
    print("n:", n, ", l:", l, ", d:", d)
    D = max([l, (n-l)//d]) + 1
    return d, l, n, D

def decideRatio(eps, d, width):
    ratio = np.exp(eps) / ((d-1) + np.exp(eps))
    print('original p=', ratio)
    integer = int(ratio * width)
    while integer > 0:
        if (width-integer) % d == 0:
            g = math.gcd(integer, width, (width-integer) // d)
            print('approximate p=', integer/width)
            return integer // g, width // g
        integer -= 1
    assert False, "Not found"    

class Prover:
    def __init__(self, data, d, l, n, D):        
        if data in CATEGORIES:
            self.data = data
        else:
            assert False, "out of categories"
        self.d, self.l, self.n, self.D = d, l, n, D
        
    def setup(self):
        print('[Prover] setup')
        
        t = self.data
        hashed_t = self.hasher.encode(t)
        self.hashed_data = hashed_t
        self.hashed_categories = list(range(self.d))
        
        print("[Prover] hased secret input: ", hashed_t, "(original: ", t, ")")
        
        mu_array = []
        mu_array = [hashed_t]*self.l
        
        for category in self.hashed_categories:
            mu_array = mu_array + ([category] * ((self.n - self.l) // self.d))
        mu_array = np.random.permutation(mu_array).tolist()
        self.mu_array = mu_array
#         print("mu array: ", mu_array)
        
    def setPubKey(self, pub_key):
        self.pub_key = pub_key

    def setHasher(self, seed):
        self.hasher = Hasher(seed, self.d)
    
    def step2(self, g_a, g_b, g_ab):
        print('[Prover] step2')
        q, g, h = self.pub_key
        w_array = []
        y_array = []
        v_array = []
        
        for i in range(0, self.n):
            r = number.getRandomRange(1, q-1)
            s = number.getRandomRange(1, q-1)
            w = pow(g, r, q) * pow(g_a, s, q) % q
            v = pow(g_b, r, q) * pow(g_ab * pow(g, i, q) % q, s, q) % q
            y = pow(g, self.D**self.mu_array[i], q) * pow(h, v, q) % q
            w_array.append(w)
            y_array.append(y)
            v_array.append(v)

        self.w_array = w_array
        self.y_array = y_array
        self.v_array = v_array

        return w_array, y_array
    
    def AKEncBool1(self):
        print('[Prover] AKEncBool1')
        q, g, h = self.pub_key
        b_array = []
        c_array = []
        s_array = []
        random_w_array = []
        for y, mu in zip(self.y_array, self.mu_array):
            random_w = number.getRandomRange(1, q-1)
            c = [0] * self.d
            s = [0] * self.d
            b = [0] * self.d
            for category in self.hashed_categories:
                if category != mu:
                    c_i = number.getRandomRange(1, q-1)
                    s_i = number.getRandomRange(1, q-1)
                    c[category] = c_i
                    s[category] = s_i
                    g_i_inv = pow(pow(g, self.D**category, q), -1, q)
                    deno = pow(y * g_i_inv % q, c_i, q)
                    b_i = pow(h, s_i, q) * pow(deno, -1, q) % q
                    b[category] = b_i
                else:
                    b_i = pow(h, random_w, q)
                    b[category] = b_i
            random_w_array.append(random_w)
            b_array.append(b)
            c_array.append(c)
            s_array.append(s)

        self.b_array = b_array
        self.s_array = s_array
        self.c_array = c_array
        self.random_w_array = random_w_array
        
        return b_array
    
    def AKEncBool3(self, ak_enc_bool_x_array):
        print('[Prover] AKEncBool3')
        q, g, h = self.pub_key
        for x,c,s,mu,v,random_w in zip(ak_enc_bool_x_array, self.c_array, self.s_array, self.mu_array, self.v_array, self.random_w_array):
            for category in self.hashed_categories:
                if category == mu:
                    c[mu] = x - sum(c)
                    s[mu] = v * c[mu] + random_w

        return self.c_array, self.s_array
    
    def AKLin1(self):
        print('[Prover] AKLin1')
        q, g, h = self.pub_key
        b_lin = [0] * self.d
        c_lin = [0] * self.d
        s_lin = [0] * self.d
        random_w = number.getRandomRange(1, q-1)
        common = sum([self.D**category for category in self.hashed_categories]) * ((self.n - self.l) // self.d)
        for category in self.hashed_categories:
            if category != self.hashed_data:
                total = common + self.l * self.D**category
                c_i = number.getRandomRange(1, q-1)
                s_i = number.getRandomRange(1, q-1)
                c_lin[category] = c_i
                s_lin[category] = s_i
                g_i_inv = pow(pow(g, total, q), -1, q)
                commitment = 1
                for y in self.y_array:
                    commitment = commitment * y % q
                deno = pow(commitment * g_i_inv % q, c_i, q)
                b_i = pow(h, s_i, q) * pow(deno, -1, q) % q
                b_lin[category] = b_i
            else:
                b_i = pow(h, random_w, q)
                b_lin[category] = b_i

        self.b_lin = b_lin
        self.s_lin = s_lin
        self.c_lin = c_lin
        self.random_w_lin = random_w
        return b_lin

    def AKLin3(self, ak_enc_bool_x_lin):
        print('[Prover] AKLin3')
        q, g, h = self.pub_key
        for category in self.hashed_categories:
            if category == self.hashed_data:
                self.c_lin[self.hashed_data] = ak_enc_bool_x_lin - sum(self.c_lin)
                v_sum = 0
                for v in self.v_array:
                    v_sum += v
                self.s_lin[self.hashed_data] = v_sum * self.c_lin[self.hashed_data] + self.random_w_lin
        return self.c_lin, self.s_lin
    
class Verifier:
    def __init__(self, d, l, n, D):
        self.d, self.l, self.n, self.D = d, l, n, D
    
    def setup(self, security):
        print('[Verifier] setup')
        q = number.getPrime(2 * security, Random.new().read)        
        g = number.getRandomRange(1, q-1)
        h = number.getRandomRange(1, q-1)
        
        seed = number.getRandomRange(1, 10000000)
        
        self.q = q
        self.g = g
        self.h = h
        self.seed = seed
        self.hasher = Hasher(seed, self.d)
        self.hashed_categories = list(range(self.d))
        
        self.sigma = np.random.randint(0, self.n)
        print("[Verifier] sigma: ", self.sigma)
        
        self.pub_key = (q, g, h)
        
    def step1(self):
        print('[Verifier] step1')
        a = number.getRandomRange(1, self.q-1)
        b = number.getRandomRange(1, self.q-1)
        self.a = a
        self.b = b
        
        g_a = pow(self.g, a, self.q)
        g_b = pow(self.g, b, self.q)
        g_ab = pow(self.g, a * b - self.sigma + 1, self.q)

        return g_a, g_b, g_ab
        
    def step3(self, w_array, y_array):
        print('[Verifier] step3')
        v_sigma = pow(w_array[self.sigma], self.b, self.q)
        g_mu_sigma = y_array[self.sigma] * pow(pow(self.h, v_sigma, self.q), -1, self.q) % self.q
        secret_output = None
        for category in self.hashed_categories:
            if pow(self.g, self.D**category, self.q) == g_mu_sigma:
                secret_output = category
        print("[Verifier] secret output (hashed): ", secret_output, "g^{mu_sigma}: ", g_mu_sigma)
        return secret_output
    
    def AKEncBool2(self, num):
        print('[Verifier] AKEncBool2')
        self.ak_enc_bool_x_array = []
        for _ in range(num):
            self.ak_enc_bool_x_array.append(number.getRandomRange(1, self.q-1))
        return self.ak_enc_bool_x_array

    def AKEncBool4(self, c_array, s_array, b_array, y_array):
        print('[Verifier] #####  AKEncBool #####')
        for s,c,b,y,x in zip(s_array, c_array, b_array, y_array, self.ak_enc_bool_x_array):
            for category in self.hashed_categories:
                if pow(self.h, s[category], self.q) != b[category] * pow(y * pow(pow(self.g, self.D**category, self.q), -1, self.q) % self.q, c[category], self.q) % self.q:
                    print("[Verifier] AKEncBool False1.")
                    return False
            if x != sum(c):
                print("[Verifier] AKEncBool False2.")
                return False
        print("[Verifier] AKEncBool OK.")
        return True 
    
    def AKLin2(self):
        print('[Verifier] AKLin2')
        self.ak_enc_bool_x_lin = 0
        self.ak_enc_bool_x_lin = number.getRandomRange(1, self.q-1)
        return self.ak_enc_bool_x_lin
    
    def AKLin4(self, s_lin, c_lin, b_lin, y_array):
        print('[Verifier] #####  AKLin #####')
        common = sum([self.D**category for category in self.hashed_categories]) * ((self.n - self.l) // self.d)
        commitment = 1
        for y in y_array:
            commitment = commitment * y % self.q
        for category in self.hashed_categories:
            total = common + self.l * self.D**category
            if pow(self.h, s_lin[category], self.q) != b_lin[category] * pow(commitment * pow(pow(self.g, total, self.q), -1, self.q) % self.q, c_lin[category], self.q) % self.q:
                print("[Verifier] AKLin False1.")
                return False
        if self.ak_enc_bool_x_lin != sum(c_lin):
            print("[Verifier] AKLin False2.")
            return False
        print("[Verifier] AKLin OK.")
        return True

In [3]:
CATEGORIES = list(range(0,100))
epsilon = 1.0
secret_input = 50
width = 1000
d_dash = 5

d, l, n, D = buildParams(epsilon, width, d_dash)

receiver = Verifier(d, l, n, D)
receiver.setup(security=80)
pub_key = receiver.pub_key
seed = receiver.seed

sender = Prover(secret_input, d, l, n, D)
sender.setPubKey(pub_key)
sender.setHasher(seed)
sender.setup()

g_a, g_b, g_ab = receiver.step1()
w_array, y_array = sender.step2(g_a, g_b, g_ab)
x = receiver.step3(w_array, y_array)

b_array = sender.AKEncBool1()
ak_enc_bool_x_array = receiver.AKEncBool2(len(y_array))
c_array, s_array = sender.AKEncBool3(ak_enc_bool_x_array)
receiver.AKEncBool4(c_array, s_array, b_array, y_array)

b_lin = sender.AKLin1()
ak_enc_bool_x_lin = receiver.AKLin2()
c_lin, s_lin = sender.AKLin3(ak_enc_bool_x_lin)
receiver.AKLin4(s_lin, c_lin, b_lin, y_array)

original p= 0.40460967519168967
approximate p= 0.4
n: 25 , l: 10 , d: 5
[Verifier] setup
[Verifier] sigma:  5
[Prover] setup
[Prover] hased secret input:  1 (original:  50 )
[Verifier] step1
[Prover] step2
[Verifier] step3
[Verifier] secret output (hashed):  None g^{mu_sigma}:  310435534034582635779259991772937314068276708061
[Prover] AKEncBool1
[Verifier] AKEncBool2
[Prover] AKEncBool3
[Verifier] #####  AKEncBool #####
[Verifier] AKEncBool OK.
[Prover] AKLin1
[Verifier] AKLin2
[Prover] AKLin3
[Verifier] #####  AKLin #####
[Verifier] AKLin OK.


True